In [6]:
import pandas as pd
import re

In [7]:
feedbacks = pd.read_csv('./old/objetivas.csv')

feedbacks.drop(columns=['curso', 'disciplina'], inplace=True)

feedbacks = feedbacks[feedbacks['Média do Professor'].notna() & feedbacks['Avaliação e feedback'].notna() & feedbacks['Aspectos operacionais'].notna() & feedbacks['Aprendizado Centrado no Aluno'].notna() & feedbacks['Ensino'].notna()]

feedbacks['responses_relative'] = feedbacks['Quantidade de formulários Respondidos'] / feedbacks['Quantidade de Alunos']

feedbacks['professor'] = feedbacks['professor'].apply(lambda x: x.upper())

feedbacks

,ano,periodo,codigo_turma,professor,final/parcial,autoavaliacao_aluno,Ensino,Aprendizado Centrado no Aluno,Aspectos operacionais,Avaliação e feedback,Avaliação da monitoria,Média,Média do Professor,Quantidade de Alunos,Quantidade de formulários Respondidos,Recomenda o professor/curso,Não recomenda o professor/curso,responses_relative
0,2015,1,CFM48COADA,ADALTO BARBACEIA GONÇALVES,F,2.91,3.27,3.32,3.63,3.50,3.00,3.30,3.41,65.0,43.0,31.0,9.0,0.661538
1,2015,1,CFM48COADA,ADALTO BARBACEIA GONÇALVES,P,2.69,3.51,3.54,3.79,3.44,3.25,3.53,3.58,65.0,51.0,48.0,2.0,0.784615
2,2015,1,CFM46GRIFA,ALEXANDRE JORGE CHAIA,F,2.95,3.29,3.25,3.10,3.23,NaN,3.27,3.22,57.0,42.0,35.0,7.0,0.736842
3,2015,1,CFM46GRIFA,ALEXANDRE JORGE CHAIA,P,2.95,3.32,3.22,3.26,3.27,3.00,3.27,3.27,57.0,42.0,37.0,3.0,0.736842
4,2015,1,CFM50MAFAA,ANGELA DE SOUZA MENEZES,F,3.48,3.35,3.40,3.58,3.40,3.70,3.38,3.43,60.0,52.0,46.0,6.0,0.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6486,2024,2,APF19SCG,JOSE CARLOS TIOMATSU OYADOMARI,F,2.70,4.00,3.70,4.00,4.00,4.00,3.85,3.91,27.0,2.0,2.0,0.0,0.074074
6487,2024,2,APFW08AMEEE,JULIANA INHASZ,F,3.49,3.81,3.75,3.30,3.58,2.79,3.78,3.63,22.0,21.0,20.0,0.0,0.954545
6489,2024,2,APF21OCAP,MARCELO DANIEL ARAUJO ERMEL,F,3.50,3.88,3.94,3.64,3.50,2.50,3.91,3.79,29.0,7.0,7.0,0.0,0.241379
6490,2024,2,PAS04EEI,RENATA DE SOUZA PRADO,F,3.00,2.24,2.64,3.55,3.20,NaN,2.44,2.81,12.0,5.0,2.0,3.0,0.416667


In [8]:

discursivas_old = pd.read_csv('./old/discursivas.csv')

grouped_continue_doing = discursivas_old.groupby(['Grupo de Turma', 'Professor', 'Avaliação'])['Disc1'].apply(lambda x: x.dropna().tolist()).reset_index()
grouped_stop_doing = discursivas_old.groupby(['Grupo de Turma', 'Professor', 'Avaliação'])['Disc2'].apply(lambda x: x.dropna().tolist()).reset_index()
grouped_start_doing = discursivas_old.groupby(['Grupo de Turma', 'Professor', 'Avaliação'])['Disc3'].apply(lambda x: x.dropna().tolist()).reset_index()

# Faz o merge com o dataframe discursivas_old
discursivas_old = pd.merge(discursivas_old, grouped_continue_doing, on=['Grupo de Turma', 'Professor', 'Avaliação'], suffixes=('', '_list'))
discursivas_old = pd.merge(discursivas_old, grouped_stop_doing, on=['Grupo de Turma', 'Professor', 'Avaliação'], suffixes=('', '_list'))
discursivas_old = pd.merge(discursivas_old, grouped_start_doing, on=['Grupo de Turma', 'Professor', 'Avaliação'], suffixes=('', '_list'))

# Remove a coluna original continue_doing
discursivas_old.drop(columns=['Disc1'], inplace=True)
discursivas_old.drop(columns=['Disc2'], inplace=True)
discursivas_old.drop(columns=['Disc3'], inplace=True)
discursivas_old.drop(columns='Família', inplace=True)

# Renomeia a coluna agrupada
discursivas_old.rename(columns={'Disc1_list': 'continue_doing'}, inplace=True)
discursivas_old.rename(columns={'Disc2_list': 'stop_doing'}, inplace=True)
discursivas_old.rename(columns={'Disc3_list': 'start_doing'}, inplace=True)
discursivas_old.rename(columns={'Grupo de Turma': 'codigo_turma'}, inplace=True)
discursivas_old.rename(columns={'Professor': 'professor'}, inplace=True)

discursivas_old['continue_doing'] = discursivas_old['continue_doing'].apply(tuple)
discursivas_old['stop_doing'] = discursivas_old['stop_doing'].apply(tuple)
discursivas_old['start_doing'] = discursivas_old['start_doing'].apply(tuple)

# Separa o ano do período
discursivas_old['Ano-Per'] = discursivas_old['Ano-Per'].apply(lambda x: str(x))
discursivas_old['ano'] = discursivas_old['Ano-Per'].apply(lambda x: x.split('.')[0])
discursivas_old['periodo'] = discursivas_old['Ano-Per'].apply(lambda x: x.split('.')[1][-1])

# Remove a coluna Ano-Per
discursivas_old.drop(columns=['Ano-Per'], inplace=True)

discursivas_old['ano'] = discursivas_old['ano'].astype(int)
discursivas_old['periodo'] = discursivas_old['periodo'].astype(int)

discursivas_old.drop_duplicates(inplace=True)
discursivas_old.sort_values('codigo_turma', inplace=True)
discursivas_old.reset_index(inplace=True, drop=True)

discursivas_old['Formulário'] = discursivas_old['Formulário'].replace({
    681: 'P',
    118: 'F'
})
discursivas_old.rename(columns={'Formulário': 'final/parcial'}, inplace=True)

discursivas_old = discursivas_old[['ano', 'periodo', 'final/parcial', 'codigo_turma', 'professor', 'continue_doing', 'stop_doing', 'start_doing']]

discursivas_old.sort_values(['professor', 'ano', 'periodo'], inplace=True)

discursivas_old

,ano,periodo,final/parcial,codigo_turma,professor,continue_doing,stop_doing,start_doing
2370,2022,3,F,LLMDT17CEET,THAIS ROMERO VEIGA SHINGAI,"(EXEMPLOS PRATICOS , OS EXERCICIOS TRAZEM UM B...","(NAO HA, NADA A ACRESCENTAR NESSE PONTO.)","(NAO HA, USAR UM METODO MAIS DIDATICO NA EXPOS..."
343,2018,1,F,CBA73AECNE,ADALTO BARBACEIA GONCALVES,(EEXPLANACAODOS ASSUNTOS CORRELATOS A DISCIPLI...,"(ALGUNS ASSUNTOS PROLIXOS., CORRER COM A MATER...",(MAIAS CALCULOS PARA AJUDAR NO DESENVOLVIMENTO...
344,2018,1,P,CBA73AECNE,ADALTO BARBACEIA GONCALVES,"(CONTINUAR A EXPLICAR DE FORMA PRATICA, SEGUIR...","(-, NADA, -, AS VEZES O PROFESSOR TRATA A MATE...","(-, NADA, -, UTILIZAR MAIS CASOS VERIDICOS E E..."
363,2018,2,P,CBA74AECNE,ADALTO BARBACEIA GONCALVES,(EXERCICIOS PRATICOS EM SALA DE AULA. EXERCICI...,"(VOTACOES NO RESPONSEWARE, NADA. O PROFESSOR T...","(TENTAR USAR CASOS REAIS., MAIS EXEMPLOS ESPEC..."
364,2018,2,F,CBA74AECNE,ADALTO BARBACEIA GONCALVES,"(TRAZENDO EXEMPLOS PRATICOS , EXPLICAR AS COIS...","(PODERIA ESCREVER MENOS NA LOUSA, UTILIZANDO M...",(PODERIA SEMPRE COMECAR A AULA FAZENDO UMA REV...
...,...,...,...,...,...,...,...,...
2825,2022,4,F,MBA8278BG_B,WALTER RUIZ JUNIOR,"(ESCLARECENDO TODAS AS DUVIDAS, EXCELENTES REF...","(NADA A DECLARAR, NADA, O SIMULADOR NAO DEVERI...","(NADA, CENTRALIZAR AS ENTREGAS NO BLACKBOARD.)"
3408,2023,2,F,MBAW2320BG,WALTER RUIZ JUNIOR,"(DANDO DICAS NA MEDIDA CERTA., PROFESSOR FOI E...","(SEM RECOMENDACOES., NADA, GOSTEI DE TUDO!, NA )","(SEM RECOMENDACOES., NADA, GOSTEI DE TUDO!)"
3441,2023,4,F,MBAW2522BG,WALTER RUIZ JUNIOR,"(EXCELENTE PROFESSOR, COM MUITA EXPERIENCIA E ...",(),()
3476,2024,2,F,MBAW2724BG,WALTER RUIZ JUNIOR,"(INTERACAO, EXEMPLOS PRATICOS E EXPLICAR A IMP...","(ACHO A DISCIPLINA INTERESSANTE, MAS A DINAMIC...","(TRABALHAR MAIS COM TEORIAS E CASES APLICADOS,..."


In [9]:
merged_surveys = pd.merge(feedbacks, discursivas_old, on=['codigo_turma', 'ano', 'periodo', 'professor', 'final/parcial'])

merged_surveys

,ano,periodo,codigo_turma,professor,final/parcial,autoavaliacao_aluno,Ensino,Aprendizado Centrado no Aluno,Aspectos operacionais,Avaliação e feedback,...,Média,Média do Professor,Quantidade de Alunos,Quantidade de formulários Respondidos,Recomenda o professor/curso,Não recomenda o professor/curso,responses_relative,continue_doing,stop_doing,start_doing
0,2018,1,LLMDSW03LLMDTW02FIN,LEONARDO PAGANO,F,3.07,3.56,3.58,3.60,3.70,...,3.57,3.59,52.0,42.0,38.0,3.0,0.807692,"(CORRECAO EXERCICIOS, EXCELENTE PROFESSOR, PRO...","(NADA, ACREDITO SER NECESSARIA UMA REFORMULACA...","(NADA, DAR EXEMPLOS PRATICOS NO MUNDO REAL REL..."
1,2018,1,MBE70MEQDA,ADRIANA BRUSCATO BORTOLUZZO,F,3.18,3.33,3.39,3.61,3.40,...,3.36,3.43,62.0,48.0,47.0,1.0,0.774194,"(CONTINUAR SENDO CLARA, OBJETIVA E EXTREMAMENT...","(NADA. EXCELENTE AULA, EXERCICIOS SIMILAR A PR...",(QUE ALINHE OS EXERCICIOS DA PROVA DE ACORDO C...
2,2018,1,MBE65GESTMKT,CARLOS ROBERTO FRANCISCO BARA,F,3.43,2.88,3.12,3.64,3.08,...,3.00,3.17,40.0,32.0,18.0,14.0,0.800000,"(DAR EXEMPLOS REAIS., UTILIZACAO DE CASES., TR...",(PASSOU MUITO SUPERFICIALMENTE POR VARIOS TEMA...,(SE APROFUNDE UM POUCO MAIS NAS DISCUSSOES DOS...
3,2018,1,MBE65ESTTECINF,EDISON AUDI KALAF,F,3.45,3.79,3.75,3.80,3.33,...,3.77,3.72,40.0,26.0,25.0,1.0,0.650000,"(ESTIMULAR A PARTICIPACAO DOS ALUNOS, BEM HUMO...",(GASTE MENOS TEMPO COM A RETOMADA DA AULA ANTE...,"(REPENSAR EM TODAS AS AULAS, NAO SOMENTE RECIC..."
4,2018,1,MBE67MBF63AMBECONGLO,EDUARDO CORREIA DE SOUZA,F,2.99,3.64,3.58,3.56,3.43,...,3.61,3.58,59.0,38.0,37.0,0.0,0.644068,"(OK, DAR MAIS BASE PARA A REALIZACAO DOS TRABA...","(OK, BOM PONTO, NADA, TIRAR SLIDES DESNECESSAR...","(OK, PASSA A TER MONITORIA., MAIS EXERCICIOS. ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,2024,2,APF19SCG,JOSE CARLOS TIOMATSU OYADOMARI,F,2.70,4.00,3.70,4.00,4.00,...,3.85,3.91,27.0,2.0,2.0,0.0,0.074074,(O PROFESSOR JOSE CARLOS SE COMUNICA DE FORMA ...,(ACHEI QUE TEMOS POUCO TEMPO DE AULA PARA ESSA...,()
1830,2024,2,APFW08AMEEE,JULIANA INHASZ,F,3.49,3.81,3.75,3.30,3.58,...,3.78,3.63,22.0,21.0,20.0,0.0,0.954545,"(CONTINUE COM A CLAREZA, DIDATICA E CONHECIMEN...","(NADA, NADA, ACHEI TUDO MUITO BOM , NADA, SEM ...",(DESENVOLVER ASSUNTOS DE FORMA MAIS COMPLEXA E...
1831,2024,2,APF21OCAP,MARCELO DANIEL ARAUJO ERMEL,F,3.50,3.88,3.94,3.64,3.50,...,3.91,3.79,29.0,7.0,7.0,0.0,0.241379,(CONTINUAR COM A PARTE PRATICA E APLICADA A ME...,"(NDA,)",(DISPONIBILIZAR UM SLIDE MESTRE COM TODOS OS T...
1832,2024,2,PAS04EEI,RENATA DE SOUZA PRADO,F,3.00,2.24,2.64,3.55,3.20,...,2.44,2.81,12.0,5.0,2.0,3.0,0.416667,"(SER GENTIL E OUVIR OS ALUNOS, CONTINUE COM A ...",(EXEMPLOS RASOS E SEM APROFUNDAMENTO/VISAO CRI...,"(PREPARO PARA AS AULAS, APROFUNDAMENTO NOS TEM..."


In [10]:
# Carrega os dados dos planos de aula
lesson_plans = pd.read_csv('../lesson-plans/planos_pgls_all.csv')

# Mantém apenas as colunas necessárias
lesson_plans = lesson_plans.drop(columns=['id'])

lesson_plans['periodo'] = lesson_plans['periodo'].apply(lambda x: str(x)[1]).astype(int)

# Função para extrair a turma
def extrair_turma_e_divisao(codigo):
    # Expressão regular para extrair a turma (letras e números iniciais)
    match_turma = re.match(r'^[A-Za-z]+\d+', codigo)
    turma = match_turma.group(0) if match_turma else codigo

    # Expressão regular para extrair a divisão (últimos caracteres após "_", se existir)
    match_divisao = re.search(r'_(\w+)$', codigo)
    divisao = match_divisao.group(1) if match_divisao else None

    # Retorna a turma com a divisão anexada, se existir
    return f"{turma}_{divisao}" if divisao else turma

turma = lesson_plans['codigo_turma'].apply(extrair_turma_e_divisao)

lesson_plans['turma'] = turma

lesson_plans['professores'] = lesson_plans['professores'].str.split(',')
lesson_plans = lesson_plans.explode('professores').reset_index(drop=True)
lesson_plans['professores'] = lesson_plans['professores'].str.strip()
lesson_plans.rename(columns={'professores': 'professor'}, inplace=True)

lesson_plans['professor'] = lesson_plans['professor'].apply(lambda x: x.upper())

lesson_plans = lesson_plans.drop(columns=['email_professores'])

lesson_plans

,ano,periodo,curso,disciplina,codigo_turma,professor,link,turma
0,2024,4,LLM - DIREITO SOCIETÁRIO,"OPERAÇÕES SOCIETÁRIAS II - OPA, RESGATE, RECESSO",LLMDS29OSIIORR,LUCIANA PEREIRA COSTA,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,LLMDS29
1,2024,4,MBA EXECUTIVO EM SAÚDE,PROJETO INTEGRADOR - INOVAÇÃO EM SAÚDE II,MBAS01PIISII,MARCUS ALEXANDRE YSHIKAWA SALUSSE,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,MBAS01
2,2024,4,MBA EXECUTIVO EM SAÚDE,PROJETO INTEGRADOR - INOVAÇÃO EM SAÚDE II,MBAS01PIISII,ISABELA BRANDÃO FURTADO,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,MBAS01
3,2024,3,LLC EM DIREITO EMPRESARIAL,"TRIBUTAÇÃO DA ATIVIDADE EMPRESARIAL (RENDA, PA...",LLC15TAERPC,DANIEL LEIB ZUGMAN,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,LLC15
4,2024,2,MASTER IN BUSINESS MANAGEMENT - ESPECIALIZAÇÃO...,ACTING IN TEAMS I,MBMTM06ATI,ANDREA LEMOS NERY,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,MBMTM06
...,...,...,...,...,...,...,...,...
6535,2018,1,MBA EXECUTIVO,MÉTODOS QUANTITATIVOS PARA TOMADA DE DECISÃO,MBE70MEQDA,ADRIANA BRUSCATO BORTOLUZZO,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,MBE70
6536,2018,1,MBA EXECUTIVO,BUSINESS DYNAMICS,MBEMOPTBUSDYN,LARS MEYER SANCHES,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,MBEMOPTBUSDYN
6537,2017,3,CBA - CERTIFICATE IN BUSINESS ADMINISTRATION,GESTÃO DE PROJETOS,CBA67GESPROJ,GUY CLIQUET DO AMARAL FILHO,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,CBA67
6538,2018,1,CBA - CERTIFICATE IN BUSINESS ADMINISTRATION,GESTÃO DE PROJETOS,CBA70GESPROJ,MARISA VILLAS BÔAS DIAS,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,CBA70


In [11]:
joined = pd.merge(lesson_plans, merged_surveys, on=['ano', 'periodo', 'codigo_turma', 'professor'])

joined.sort_values(['ano', 'periodo'], ascending=False, inplace=True)

joined = joined[
    [
        'ano', 
        'periodo', 
        'curso',
        'disciplina',
        'turma',
        'codigo_turma',
        'professor',
        'final/parcial',
        'link',
        'autoavaliacao_aluno',
        'Ensino',
        'Aprendizado Centrado no Aluno',
        'Aspectos operacionais',
        'Avaliação e feedback',
        'Avaliação da monitoria',
        'Média',
        'Média do Professor',
        'Quantidade de Alunos',
        'Quantidade de formulários Respondidos',
        'responses_relative',
        'Recomenda o professor/curso',
        'Não recomenda o professor/curso',
        'continue_doing',
        'stop_doing',
        'start_doing'
    ]
]

joined.sort_values(['ano', 'periodo', 'professor', 'turma'], ascending=False, inplace=True)

joined.reset_index(drop=True, inplace=True)

joined

,ano,periodo,curso,disciplina,turma,codigo_turma,professor,final/parcial,link,autoavaliacao_aluno,...,Média,Média do Professor,Quantidade de Alunos,Quantidade de formulários Respondidos,responses_relative,Recomenda o professor/curso,Não recomenda o professor/curso,continue_doing,stop_doing,start_doing
0,2024,2,MBA EXECUTIVO,BUSINESS GAME,MBAW2724,MBAW2724BG,WALTER RUIZ JUNIOR,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,3.58,...,3.85,3.84,19.0,18.0,0.947368,16.0,2.0,"(INTERACAO, EXEMPLOS PRATICOS E EXPLICAR A IMP...","(ACHO A DISCIPLINA INTERESSANTE, MAS A DINAMIC...","(TRABALHAR MAIS COM TEORIAS E CASES APLICADOS,..."
1,2024,2,CFM - CERTIFICATE IN FINANCIAL MANAGEMENT,DECISÕES FINANCEIRAS EMPRESARIAIS,CFM81,CFM81DFE,VALERIO MACHADO DALLOLIO,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,3.23,...,3.81,3.85,26.0,16.0,0.615385,15.0,1.0,(O MELHOR PARA O FINAL. UM CRAQUE DAS SALAS DE...,(O MELHOR PARA O FINAL. UM CRAQUE DAS SALAS DE...,(O MELHOR PARA O FINAL. UM CRAQUE DAS SALAS DE...
2,2024,2,MBA EXECUTIVO INTERNACIONAL,NEGÓCIOS INOVADORES II - EXPERIÊNCIA INTEGRADORA,MBAEXIW04,MBAEXIW04NIIIEI,THIAGO DE CARVALHO,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,3.84,...,1.93,2.29,11.0,10.0,0.909091,1.0,9.0,"(E IMPORTANTE CONTINUAR A TRAZER TEORIA, POREM...","(O PROFESSOR E CONFUSO, NAO PARECE DOMINAR OS ...","(E IMPORTANTE CONTINUAR A TRAZER TEORIA, POREM..."
3,2024,2,MBA EXECUTIVO EM SAÚDE,MARKETING PARA GESTORES EM SAÚDE,MBAS01,MBAS01JEP,SILVIA ANTONIO SFEIR,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,3.30,...,3.50,3.50,12.0,2.0,0.166667,2.0,0.0,"(EXEMPLOS PRATICOS E EXERCICIOS EM CONJUNTO , ...",(FOCAR EM ESTUDAR A MATERIA OFF-LINE PARA PROV...,(SERIA INTERESSANTE FAZER UMA DINAMICA EM GRUP...
4,2024,2,MASTER IN BUSINESS MANAGEMENT - ESPECIALIZAÇÃO...,PROJECT DELIVERY - AGILE: CLASSIC: HIBRID,MBMPM02,MBMPM02PDACH,RONNIE MASCHK,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,3.06,...,3.11,2.99,13.0,10.0,0.769231,9.0,1.0,"(DANDO EXEMPLOS, SER GENTIL E DISPONIVEL PARA ...",(ESPERAR TANTO TEMPO PELA RESPOSTAS DOS ALUNOS...,"(APROFUNDAR MAIS O CONTEUDO, TRAZER MAIS EXEMP..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357,2018,1,MBA EXECUTIVO,AMBIENTE ECONÔMICO GLOBAL,MBE67,MBE67MBF63AMBECONGLO,EDUARDO CORREIA DE SOUZA,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,2.99,...,3.61,3.58,59.0,38.0,0.644068,37.0,0.0,"(OK, DAR MAIS BASE PARA A REALIZACAO DOS TRABA...","(OK, BOM PONTO, NADA, TIRAR SLIDES DESNECESSAR...","(OK, PASSA A TER MONITORIA., MAIS EXERCICIOS. ..."
1358,2018,1,MBA EXECUTIVO,ESTRATÉGIA DE TECNOLOGIA DA INFORMAÇÃO,MBE65,MBE65ESTTECINF,EDISON AUDI KALAF,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,3.45,...,3.77,3.72,40.0,26.0,0.650000,25.0,1.0,"(ESTIMULAR A PARTICIPACAO DOS ALUNOS, BEM HUMO...",(GASTE MENOS TEMPO COM A RETOMADA DA AULA ANTE...,"(REPENSAR EM TODAS AS AULAS, NAO SOMENTE RECIC..."
1359,2018,1,MBA EXECUTIVO,GESTÃO DE MARKETING,MBE65,MBE65GESTMKT,CARLOS ROBERTO FRANCISCO BARA,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,3.43,...,3.00,3.17,40.0,32.0,0.800000,18.0,14.0,"(DAR EXEMPLOS REAIS., UTILIZACAO DE CASES., TR...",(PASSOU MUITO SUPERFICIALMENTE POR VARIOS TEMA...,(SE APROFUNDE UM POUCO MAIS NAS DISCUSSOES DOS...
1360,2018,1,PROGRAMA AVANÇADO EM GESTÃO PÚBLICA,CONSTITUIÇÃO DO ESTADO E POLÍTICA,PAGPW01,PAGPW01CONSESTP,CARLOS ALBERTO FURTADO DE MELO,F,https://cgi.insper.edu.br/DiarioDeClasse/Plano...,3.28,...,2.66,2.81,32.0,21.0,0.656250,14.0,5.0,(CLARAMENTE O PROFESSOR CONHECE MUITO DO TEMA....,(A PROPOSTA DA DISCIPLINA SER LECIONADA POR DO...,"(CASO A AULA CONTINUE SENDO LECIONADA EM PAR, ..."
